In [1]:
import sagemaker.huggingface
import sagemaker
from transformers import AutoTokenizer

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::448807757624:role/service-role/AmazonSageMaker-ExecutionRole-20211202T101582
sagemaker bucket: sagemaker-us-east-2-448807757624
sagemaker session region: us-east-2


In [19]:
s3_prefix = 'datasets/xsum_corrupted_predicateonly'
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'

In [22]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 4,
                 'train_batch_size': 1,
                 'model_name':'facebook/bart-base',
                 'output_dir':'/opt/ml/checkpoints'
                 }

# s3 uri where our checkpoints will be uploaded during training
job_name = "xsum-predicate-bart-base"
checkpoint_s3_uri = f's3://{sess.default_bucket()}/{job_name}/checkpoints'

In [25]:
huggingface_estimator = HuggingFace(entry_point='train_xsum.py',
                            source_dir='./scripts',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            base_job_name=job_name,
                            checkpoint_s3_uri=checkpoint_s3_uri,
#                             use_spot_instances=True,
#                             max_wait=25200, # This should be equal to or greater than max_run in seconds'
                            max_run=100000, # expected max run in seconds
                            role=role,
                            transformers_version='4.6',
                            pytorch_version='1.7',
                            py_version='py36',
                            hyperparameters = hyperparameters)

In [26]:
huggingface_estimator.fit({'train': training_input_path})

2021-12-08 00:26:22 Starting - Starting the training job...
2021-12-08 00:26:46 Starting - Launching requested ML instancesProfilerReport-1638923182: InProgress
...
2021-12-08 00:27:08 Failed - Training job failed
..

UnexpectedStatusException: Error for Training job xsum-predicate-bart-base-2021-12-08-00-26-22-315: Failed. Reason: CapacityError: Unable to provision requested ML compute capacity. Please retry using a different ML instance type.